# Julia's Type System

Julia programs manipulate *values*, and every value has two parts: a *type* part and a data part. The type part answers the question "what kind of thing is this?", and the data part distinguishes one thing of a certain kind from every other thing of that kind.

## Part 1. DataType

In [42]:
typeof(3)

Int64

In this case the type is `Int64` and the data part is the bits `...0011`.

In Julia types are also values:

In [2]:
typeof(Int64)

DataType

In [3]:
typeof(DataType)

DataType

In fact, the identity `typeof(typeof(x)) === DataType` holds for all values in Julia. `DataType` is the backbone of the entire system. It does many jobs, which can be identified by looking inside a `DataType` object:

### DataType Job 1: A symbolic description

This consists of a name (which is mostly a string), and a vector of sub-components:

In [14]:
T = typeof(1+2im)

Complex{Int64}

In [15]:
T.name

typename(Complex)

In [16]:
T.parameters

svec(Int64)

### DataType Job 2: A nominal hierarchy of types

DataTypes form a tree of declared type relationships ("an x is-a y"):

In [17]:
T.super

Number

In [18]:
T.super.super.super.super  # `Any` is the built-in top of the hierarchy.

Any

### DataType Job 3: Describe the representation

In [19]:
T.types

svec(Int64, Int64)

In [20]:
T.name.names

svec(:re, :im)

In [21]:
T.size

ErrorException: type DataType has no field size

In [22]:
T.mutable   # whether this was declared with `type` (vs. `immutable`)

ErrorException: type DataType has no field mutable

In [23]:
T.abstract  # whether this was declared with `abstract`

ErrorException: type DataType has no field abstract

In [32]:
T.ninitialized

ErrorException: type DataType has no field initialized

In [25]:
T.layout

Ptr{Nothing} @0x00007ffa02c33bf0

### Defining struct types


In [26]:
struct Point
    x::Float64
    y::Float64
end

In [27]:
Point(1,2)

Point(1.0, 2.0)

### Abstract vs. Concrete

`abstract` types can have declared subtypes, while concrete types can have instances. These are separated because if an `X` IS-A `Y`, and `Y` specifies a representation, then `X` had better have the same representation.

"car is-a vehicle" is correct because "vehicle" is an abstract concept that doesn't commit to any specifics. But if I tell you I'm giving you a Porsche, it needs to look like a Porsche.

A type `T` is concrete if there could be some value `x` such that `typeof(x) === T`. This is also sometimes called a "leaf type".

In [28]:
abstract type PointLike end

# struct Point <: PointLike

## Part 2. Type parameters

Type parameters can be completely or partially specified:

In [29]:
Array{Int}

Array{Int64}

In [30]:
[1] isa Array

true

In [31]:
Array{Int,2}

Matrix{Int64} (alias for Array{Int64, 2})

A type is concrete (can have instances) if
    1. it is not declared `abstract`
    2. all parameters are specified

In [33]:
[1] isa Array{Int,1}

true

In [34]:
[1] isa Array{Int}

true

In [35]:
[1] isa Array{Number}

false

In [36]:
Int <: Number

true

Types with different *specified* parameters are just different, and have no subtype relationship. This is called *invariance*.

### Defining types with parameters

In [37]:
struct GenericPoint{T<:Real}
    x::T
    y::T
end

In [38]:
GenericPoint(1,2)

GenericPoint{Int64}(1, 2)

In [39]:
GenericPoint(1.0,2.0)

GenericPoint{Float64}(1.0, 2.0)

In [40]:
GenericPoint(1,2.0)

MethodError: MethodError: no method matching GenericPoint(::Int64, ::Float64)

Closest candidates are:
  GenericPoint(::T, !Matched::T) where T<:Real
   @ Main c:\Users\jwei\Documents\GitHub\Julia\JuliaTutorials\introductory-tutorials\intro-to-julia\long-version\120 Types.ipynb:2


### Tuple types

In [41]:
typeof((1,2.0))

Tuple{Int64, Float64}

Very similar to other DataTypes, except
    1. Have no field names, only indices
    2. `T.parameters == T.types`
    3. Are always immutable
    4. Can have any number of fields

These factors conspire to make Tuples the only *covariant* types in Julia:

In [44]:
Tuple{Int} <: Tuple{Number}

true

A Tuple type is concrete iff all its field types are.

Tuple types can be abstract with respect to the number of elements. These are called variadic tuple types, or vararg types.

In [45]:
Tuple{Int,Vararg{Int}}

Tuple{Int64, Vararg{Int64}}

Note that `Vararg` refers to the tail of a tuple type, and as such is not a first-class type itself. It only makes sense inside a Tuple type. This is a bit unfortunate.

The second parameter to `Vararg` is a length, which can also be either unspecified (as above), or specified:

In [46]:
Tuple{Int,Vararg{Int,2}}

Tuple{Int64, Int64, Int64}

## Part 3. Larger type domains

### Union types

A type can be thought of as a set of possible values. A type expresses *uncertainty* about which value we have. You can do set operations on them.

In [47]:
Union{Int64,Float64}

Union{Float64, Int64}

In [48]:
1 isa Union{Int64,Float64}

true

In [49]:
Int64 <: Number

true

In [50]:
Int64 <: Union{Int64,Float64}

true

In [51]:
Union{Int,String} <: Union{Int,String,Float32}

true

In [52]:
typeintersect(Union{Int,String}, Union{Int,String,Float32})

Union{Int64, String}

Union types naturally lend themselves to missing data.

In [53]:
data = [1.1, missing, 3.2, missing, 5.7, 0.4]

6-element Vector{Union{Missing, Float64}}:
 1.1
  missing
 3.2
  missing
 5.7
 0.4

### UnionAll types

This is an *iterated union* of types.

`Array{T,1} where T<:Integer`

Means "the union of all types of the form Array{T,1} where T is a subtype of Integer".

This expresses uncertainty about the value of a parameter.

This concept exists in all versions of Julia, but does not have syntax or fully correct support within the system until upcoming v0.6.0 (currently on branch jb/subtype).

* Since this kind of type introduces *variables*, its expressive power is (probably) equivalent to quantified boolean formulas.
* Requires a quantified-SAT solver in the compiler.
* Under common assumptions, harder than NP-complete.

In [57]:
# this definition is in the Base library
Vector = Array{T,1} where T

ErrorException: cannot assign a value to imported variable Base.Vector from module Main

These are used to express "unspecified parameters".

These also describe methods with "method parameters" (or "static parameters"):

In [58]:
func(a::Array{T,1}) where {T<:Integer} = T

func (generic function with 1 method)

In [59]:
func([0x00])

UInt8

In [60]:
func([1.0])

MethodError: MethodError: no method matching func(::Vector{Float64})

Closest candidates are:
  func(!Matched::Vector{T}) where T<:Integer
   @ Main c:\Users\jwei\Documents\GitHub\Julia\JuliaTutorials\introductory-tutorials\intro-to-julia\long-version\120 Types.ipynb:1


#### Question

What is the difference between

`Vector{Vector{T}} where T`

and

`Vector{Vector{T} where T}`?

Is one a subtype of the other?

## Answer

See https://discourse.julialang.org/t/vector-vector-t-where-t-vs-vector-vector-t-where-t/2743

Suppose there were only two types in Julia such that `T` is either `String` or `Int`.

Then the first is equivalent to

`Vector{Union{Vector{String}, Vector{Int}}}`

And the second is equivalent to

`Union{Vector{Vector{String}}, Vector{Vector{Int}}}`

### Exercise

Define a `UnitPoint{<:Real}` parametric struct type which has `x` and `y` fields of type `T` and which has an inner constructor that normalizes its arguments by diving them by `hypot(x, y)` upon construction, guaranteeing that the resulting point is on the unit circle.

In [83]:
struct UnitPoint{T<:Real}
    x::T
    y::T
    function UnitPoint{T}(x, y) where {T <: Real}
        distance = hypot(x, y)
        return new(x / distance, y / distance)
    end
end

In [84]:
a = UnitPoint{Float64}(1, 2)
a.x

0.4472135954999579